In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [5]:
file = '311_Service_Requests_from_2010_to_Present.csv'

In [16]:
#print(pd.read_csv(file, nrows=5))
test = pd.read_csv(file, nrows=5)
test = test.rename(columns={c: c.replace(' ', '') for c in test.columns}) 
test = test.rename(columns={c: c.replace('(', '') for c in test.columns})
test = test.rename(columns={c: c.replace(')', '') for c in test.columns})


In [17]:
test.columns = [c.lower() for c in test.columns]
test

,uniquekey,createddate,closeddate,agency,agencyname,complainttype,descriptor,locationtype,incidentzip,incidentaddress,...,vehicletype,taxicompanyborough,taxipickuplocation,bridgehighwayname,bridgehighwaydirection,roadramp,bridgehighwaysegment,latitude,longitude,location
0,24811305,01/18/2013 12:00:00 AM,02/11/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PLUMBING,BASIN/SINK,RESIDENTIAL BUILDING,11219,1454 43 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.636336,-73.985246,"(40.63633580546793, -73.98524566850578)"
1,24811306,01/18/2013 12:00:00 AM,01/23/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,FLOOR,RESIDENTIAL BUILDING,10468,2420 DAVIDSON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.861985,-73.902833,"(40.86198534161541, -73.90283327659593)"
2,24811307,01/18/2013 12:00:00 AM,01/18/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10025,858 WEST END AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.798897,-73.970257,"(40.79889676055833, -73.97025674483577)"
3,24811308,01/18/2013 12:00:00 AM,01/23/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PLUMBING,BASIN/SINK,RESIDENTIAL BUILDING,11206,160 VERNON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.695054,-73.945363,"(40.69505379327949, -73.94536269567737)"
4,24811309,01/18/2013 12:00:00 AM,01/23/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,11233,841 HALSEY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.685352,-73.919075,"(40.68535201436509, -73.91907456016442)"


In [18]:
test.columns

Index(['uniquekey', 'createddate', 'closeddate', 'agency', 'agencyname',
       'complainttype', 'descriptor', 'locationtype', 'incidentzip',
       'incidentaddress', 'streetname', 'crossstreet1', 'crossstreet2',
       'intersectionstreet1', 'intersectionstreet2', 'addresstype', 'city',
       'landmark', 'facilitytype', 'status', 'duedate',
       'resolutiondescription', 'resolutionactionupdateddate',
       'communityboard', 'bbl', 'borough', 'xcoordinatestateplane',
       'ycoordinatestateplane', 'opendatachanneltype', 'parkfacilityname',
       'parkborough', 'vehicletype', 'taxicompanyborough',
       'taxipickuplocation', 'bridgehighwayname', 'bridgehighwaydirection',
       'roadramp', 'bridgehighwaysegment', 'latitude', 'longitude',
       'location'],
      dtype='object')

In [6]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'Health_Inspection'
username = 'postgres' # change this to your username

In [7]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://postgres@localhost/Health_Inspection


In [8]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


# Get data into postgres

In [27]:
chunksize = 100000
i = 0
j = 1
#colnames = ['uniquekey', 'createddate', 'closeddate', 'agency', 'agencyname','complainttype', 'descriptor', 'locationtype', 'incidentzip','incidentaddress', 'streetname', 'crossstreet1', 'crossstreet2','intersectionstreet1', 'intersectionstreet2', 'addresstype', 'city','landmark', 'facilitytype', 'status', 'duedate','resolutiondescription', 'resolutionactionupdateddate','communityboard', 'bbl', 'borough', 'xcoordinatestateplane','ycoordinatestateplane', 'opendatachanneltype', 'parkfacilityname','parkborough', 'vehicletype', 'taxicompanyborough','taxipickuplocation', 'bridgehighwayname', 'bridgehighwaydirection','roadramp', 'bridgehighwaysegment', 'latitude', 'longitude','location']
for df in pd.read_csv(file, chunksize=chunksize, iterator=True, low_memory=False):
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
    df = df.rename(columns={c: c.replace('(', '') for c in df.columns})
    df = df.rename(columns={c: c.replace(')', '') for c in df.columns})
    df.columns = [c.lower() for c in df.columns]
    df.index += j
    i+=1
    df.to_sql('ny311table2', engine, if_exists='append')
    j = df.index[-1] + 1

# Use SQL within python

In [28]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)


In [36]:
# query:
sql_query = """
SELECT createddate, complainttype, descriptor,locationtype, incidentzip, landmark, latitude,longitude,(SUBSTR(createddate, 7, 4) || '-' || LEFT(createddate, 2) || '-' || SUBSTR(createddate, 4, 2))::timestamp AS cleaned_date
FROM ny311table2 
WHERE locationtype IN ('Club/Bar/Restaurant','Restaurant/Bar/Deli/Bakery','Restaurant','Food Cart Vendor', 'Catering Service','Cafeteria- College/University','Cafeteria-Public School', 'Cafeteria- Private School','Cafeteria- Private', 'Cafeteria', 'Catering Hall','Bar/Restaurant')
;
"""
health_data_from_sql = pd.read_sql_query(sql_query,con)
health_data_from_sql.head()
len(health_data_from_sql)

236692

In [37]:
test = health_data_from_sql[health_data_from_sql['cleaned_date']>= '2014-01-01']

In [39]:
test.to_csv(r'NY311_part2.csv')

In [38]:
len(test)

140042

# Put week 2 dataset into SQL

In [2]:
file2 = 'DataforWk2.csv'

In [6]:
cd '/Users/rabarry/Documents/Insight/NYC data'

/Users/rabarry/Documents/Insight/NYC data


In [3]:
# Set your postgres username
dbname = 'Health_Inspection'
username = 'postgres' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://postgres@localhost/Health_Inspection


In [7]:
df = pd.read_csv(file2)

In [8]:
df.to_sql('DataforWk2', engine, if_exists='append')

In [9]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)

In [44]:
rest_name = 'starbucks'
zip_code = int('10001')

In [45]:
# query:
query = "SELECT \"DBA\", ""\"BORO_cat\",\"CUISINE DESCRIPTION_cat\",\"Second_Last_Insp_Num_CriticialFlags\",\"Second_Last_Insp_Type_cat\",\"Insp_Date_Diff\", \"num_years_active\"  FROM \"DataforWk2\" WHERE \"DBA\" ILIKE '%s' AND \"ZIPCODE\"=%d" %(rest_name, zip_code)
print(query)
query_results=pd.read_sql_query(query,con)
print(query_results)
births = []

SELECT "DBA", "BORO_cat","CUISINE DESCRIPTION_cat","Second_Last_Insp_Num_CriticialFlags","Second_Last_Insp_Type_cat","Insp_Date_Diff", "num_years_active"  FROM "DataforWk2" WHERE "DBA" ILIKE 'starbucks' AND "ZIPCODE"=10001
          DBA  BORO_cat  CUISINE DESCRIPTION_cat  \
0   STARBUCKS         2                       13   
1   STARBUCKS         2                       13   
2   STARBUCKS         2                       13   
3   STARBUCKS         2                       13   
4   STARBUCKS         2                       13   
5   STARBUCKS         2                       13   
6   STARBUCKS         2                       13   
7   STARBUCKS         2                       13   
8   STARBUCKS         2                       13   
9   STARBUCKS         2                       13   
10  STARBUCKS         2                        2   
11  STARBUCKS         2                       13   

    Second_Last_Insp_Num_CriticialFlags  Second_Last_Insp_Type_cat  \
0                             

In [39]:
query_results

,BORO_cat,CUISINE DESCRIPTION_cat,Second_Last_Insp_Num_CriticialFlags,Second_Last_Insp_Type_cat,Insp_Date_Diff,num_years_active
0,2,44,1.0,0,386.0,2.0


In [29]:
for i in range(0,query_results.shape[0]):
    births.append(dict(index=query_results.iloc[i]['index'], attendant=query_results.iloc[i]['attendant'], birth_month=query_results.iloc[i]['birth_month']))
    the_result = ModelIt(patient,births)
    return render_template("output.html", births = births, the_result = the_result)

1

In [30]:
health_data_from_sql

,BORO_cat,CUISINE DESCRIPTION_cat,Second_Last_Insp_Num_CriticialFlags,Second_Last_Insp_Type_cat,Insp_Date_Diff,num_years_active
0,2,44,1.0,0,386.0,2.0


In [40]:
df.iloc[7]

Unnamed: 0                                                 7
CAMIS                                               40357217
Second_Last_Insp_Type                                Initial
Second_Last_Insp_Date                             2018-05-31
Second_Last_Insp_Num_CriticialFlags                        0
Last_Insp_Type                                       Initial
Last_Insp_Date                                    2019-05-22
Last_Insp_Num_CriticialFlags                               1
Insp_Date_Diff                                           356
DBA                                                WILD ASIA
BORO                                                   Bronx
ZIPCODE                                                10460
CUISINE DESCRIPTION                                 American
Latitude                                             40.8506
Longitude                                           -73.8825
Community Board                                          206
Council District        